In [102]:
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmreadout.structures as structures

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
array = np.random.rand(400,20)
array = np.zeros((400,20), dtype=np.float32)

ds = structures.DataSeries(array, dimensions=(40,10))

ds.crop(xrange=(0,10), yrange=(0,20))

ds.mapview[1,1,2]=2

ds.data[11,2]


2.0

In [113]:
array = np.random.randint(0,100,(400,20)).astype(np.float32)
array_se = np.sqrt(array)

In [118]:
dsd = structures.DataSeries(array, dimensions=(40,10))

dss = structures.DataSeries(array_se, dimensions=(40,10))

ds = structures.DataSet(dsd, dss)

ds.se.data

array([[9.69536  , 6.3245554, 9.848858 , ..., 7.28011  , 3.       ,
        0.       ],
       [1.7320508, 9.848858 , 8.717798 , ..., 7.8740077, 7.483315 ,
        9.848858 ],
       [8.6602545, 3.1622777, 6.164414 , ..., 9.055386 , 9.       ,
        3.3166249],
       ...,
       [9.539392 , 7.81025  , 9.899495 , ..., 9.949874 , 7.615773 ,
        4.582576 ],
       [5.2915025, 7.6811457, 2.236068 , ..., 9.848858 , 6.5574384,
        6.6332498],
       [1.4142135, 9.949874 , 5.0990195, ..., 8.426149 , 8.426149 ,
        9.643651 ]], dtype=float32)